<a href="https://colab.research.google.com/github/sizah123/myApplication/blob/master/sizah.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get --purge remove cuda nvidia* libnvidia-*


In [0]:
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge


In [0]:
!apt-get remove cuda-*


In [0]:
!apt autoremove


In [0]:
!apt-get update

In [0]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb


In [0]:
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb


In [0]:
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub


In [0]:
!apt-get update


In [0]:
!apt-get install cuda-9.2

In [0]:
!nvcc --version

In [0]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

In [0]:
pwd


In [0]:
!nvcc -o square ./square.cu

In [0]:
!./square

In [0]:
%load_ext nvcc_plugin

In [0]:
!wget "https://www.duckietown.org/wp-content/uploads/2018/05/duckietown_nice-1024x683.jpg" -O dt.jpg


In [0]:
# np = common abbreviation for numpy
import numpy as np

# Constructing a numpy array from a list

boring_list = [1, 2, 3]
fun_array = np.array(boring_list) # Calling np.array(a_list) will cast your list into a numpy array!

# Arrays can be added, and there are tons of convenient functions to make them
ones = np.ones(3)
result = fun_array + ones + np.zeros(3)

print(result)

# What about matrices?
boring_mtx = [[1, 2], [3, 4]]
ones = np.ones((2, 2)) # (2, 2) describes the shape that I want
identity = np.eye(2) # an Identity matrix

# Numpy will cast your lists into arrays if need be
try:
    result = boring_mtx + identity 
    print(result)
except:
    print("Can I do this?")
    
# Random Sampling? No problem!
rando_mtx = np.random.random((100, 100))

# A lot of functions built in
print(np.linalg.norm(rando_mtx))

# But different behavior depending on the parameters you give it!
print(np.linalg.norm(rando_mtx, axis=0))
print(np.linalg.norm(rando_mtx, ord=2))

# Matrix Operations work exactly how you think they would
rando_mtx = np.eye(rando_mtx.shape[0]) * rando_mtx


# Or not? One big (and subtle) feature: broadcasting
rando_mtx += 1 # adding a scalar works, that's cool! 

mtx1 = np.random.random((8, 1, 7, 1))
mtx2 = np.random.random((8, 2, 1, 9))

# No error? 
res = mtx1 * mtx2 

# Does the same? 
res2 = mtx1.dot(mtx2)

print(np.array_equal(res, res2), res.shape, res2.shape)

# Big takeaway: numpy docs are your friend! Look before you write!


In [0]:
%matplotlib inline
from matplotlib import pyplot as plt
import cv2

# Load in Grayscale
img = cv2.imread('./dt.jpg', 0)

# The underlying representation is a numpy array!
print(type(img))

plt.imshow(img)
plt.show()

img = cv2.imread('./dt.jpg')

plt.imshow(img)
plt.show()

# This would work normally, but it will crash the colab kernel, so don't.
# cv2.imshow('Image', img)


In [0]:
imgrgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

plt.imshow(imgrgb)
plt.show()

# Or, use fun numpy functions / indexing!
imgrgb = img[:,:,::-1]
plt.imshow(imgrgb)
plt.show()


In [0]:
# Expanding
res = cv2.resize(imgrgb, None,fx=2, fy=2, interpolation = cv2.INTER_CUBIC)
plt.imshow(res)
plt.show()

# Shrinking
res = cv2.resize(imgrgb, None,fx=.1, fy=.1, interpolation = cv2.INTER_CUBIC)
plt.imshow(res)
plt.show()

# What are some practical constraints that have to do with image resizing?

In [0]:
#@title Transformation Parameters { run: "auto" }

theta_degrees = 0 #@param {type:"slider", min:0, max:360, step:10}
shift_x = 100 #@param {type:"slider", min:-100, max:100, step:2}
shift_y = 100 #@param {type:"slider", min:-100, max:100, step:2}

# We can also play around with rotations by defining our M matrix, 
# which has the form:
"""
| cos(theta) -sin(theta) tx | 
| sin(theta)  cos(theta) ty |
"""

rows, cols, _ = imgrgb.shape

radians = theta_degrees * np.pi / 180
M = [
        [np.cos(radians), -np.sin(radians), shift_x], 
        [np.sin(radians),  np.cos(radians), shift_y]
    ]

M = np.array(M)
rows += int(shift_x)
cols += int(shift_y)

res = cv2.warpAffine(imgrgb, M, (cols,rows))

plt.imshow(res)
plt.show()




In [0]:
# make things easier!
img = imgrgb

# Thresholding is cool
ret,thresh1 = cv2.threshold(img,127,255,cv2.THRESH_BINARY)
ret,thresh2 = cv2.threshold(img,127,255,cv2.THRESH_BINARY_INV)
ret,thresh3 = cv2.threshold(img,127,255,cv2.THRESH_TRUNC)
ret,thresh4 = cv2.threshold(img,127,255,cv2.THRESH_TOZERO)
ret,thresh5 = cv2.threshold(img,127,255,cv2.THRESH_TOZERO_INV)

titles = ['Original Image','BINARY','BINARY_INV','TRUNC','TOZERO','TOZERO_INV']
images = [img, thresh1, thresh2, thresh3, thresh4, thresh5]

for i in range(6):
    plt.subplot(2,3,i+1),plt.imshow(images[i],'gray')
    plt.title(titles[i])
    plt.xticks([]),plt.yticks([])

plt.show()

# Blurring - smoothes the image out
blur = cv2.blur(img,(10, 10))
gblur = cv2.GaussianBlur(imgrgb,(5,5),0)

plt.imshow(img), plt.title('Original')
plt.show()
plt.imshow(blur), plt.title('Blurred')
plt.show()
plt.imshow(gblur),plt.title('Gaussian Blur')
plt.show()
